In [1]:
#news url crawling
import requests
import pandas as pd
from bs4 import BeautifulSoup

def make_urllist(page_num, code, date):
    urllist = []
    for i in range(1, page_num+1) :
        url = 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1='+str(code)+'&date='+str(date)+'&page='+str(i)
        news = requests.get(url)
        
        soup = BeautifulSoup(news.content, 'html.parser')
        
        news_list = soup.select('.newsflash_body .type06_headline li dl')
        news_list.extend(soup.select('.newsflash_body .type06 li dl'))
        
        for line in news_list:
            urllist.append(line.a.get("href"))
    return urllist

In [64]:
#number code to English word
idx2word = {'100':'Urgent',
            '101':'Economy',
            '102':'Society',
            '103':'Life/Culture',
            '104':'World',
            '105' : 'IT/Science'
           }

In [3]:
#make data
from newspaper import Article

def make_data(urllist, code):
    text_list = []
    for url in urllist:
        article = Article(url, language='ko')
        article.download()
        article.parse()
        text_list.append(article.text)
        
    df = pd.DataFrame({'news':text_list})
        
    df['code'] = idx2word[str(code)]
    return df

In [57]:
code_list = [100, 101, 102, 103, 104, 105]

code_list

[100, 101, 102, 103, 104, 105]

In [65]:
#totally make data
def make_total_data(page_num, code_list, date):
  df = None

  for code in code_list:
    url_list = make_urllist(page_num, code, date)
    df_temp = make_data(url_list, code)
    print(str(code)+'번 코드에 대한 데이터를 만들었습니다.')

    if df is not None:
      df = pd.concat([df, df_temp])
    else:
      df = df_temp

  return df

In [79]:
df = make_total_data(3, code_list, 20200506)

100번 코드에 대한 데이터를 만들었습니다.
101번 코드에 대한 데이터를 만들었습니다.
102번 코드에 대한 데이터를 만들었습니다.
103번 코드에 대한 데이터를 만들었습니다.
104번 코드에 대한 데이터를 만들었습니다.
105번 코드에 대한 데이터를 만들었습니다.


In [80]:
#only valid letter
df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['news']

0     안철수 국민의당 대표가 지난 일 서울 영등포구 이룸센터에서 열린 혁신준비위원회 제차...
1     동영상 뉴스이재용 삼성전자 부회장이 경영권 승계와 노조 문제 등과 관련해 대국민사과...
2     서훈 국가정보원장이 일 국회 정보위원회 전체회의에 참석해 자리에 앉고 있다 권도현 ...
3     미 북한 전문 사이트 분단을 넘어는 일 북한이 평양 순안국제공항 인근 신리에서 탄도...
4     김연철 통일부 장관이 북한군의 남측 감시초소 총격 사건이 발생한 지 사흘 만인 일 ...
                            ...                        
55    포항에 위치한 방사광가속기 모습 포항가속기연구소 제공 포항에 위치한 방사광가속기 모...
56    게티이미지뱅크 게티이미지뱅크  전자신문  전자신문인터넷 무단전재 및 재배포 금지통신...
57    일 오후 대전 유성구 인터시티호텔에서 열린 산업지원 다목적 방사광가속기 구축 부지선...
58    산업지원 다목적 방사광가속기 구축 부지선정평가위원회는 일 개 후보 지자체에 대한 발...
59    대전뉴스 김기태 기자  일 오후 대전 유성구 인터시티호텔에서 열린 산업지원 다목적 ...
Name: news, Length: 360, dtype: object

In [81]:
#is there null data?
print(df.isnull().sum())

#drop_duplicates
df.drop_duplicates(subset=['news'], inplace=True)
print('뉴스 기사의 개수 :', len(df))

news    0
code    0
dtype: int64
뉴스 기사의 개수 : 278


In [ ]:
from konlpy.tag import Mecab
tokenizer = Mecab()

kor_text = '밤에 귀가하던 여성에게 범죄를 시도한 대 남성이 구속됐다서울 제주경찰서는 \
            상해 혐의로 씨를 구속해 수사하고 있다고 일 밝혔다씨는 지난달 일 피해 여성을 \
            인근 지하철 역에서부터 따라가 폭행을 시도하려다가 도망간 혐의를 받는다피해 \
            여성이 저항하자 놀란 씨는 도망갔으며 신고를 받고 주변을 수색하던 경찰에 \
            체포됐다피해 여성은 이 과정에서 경미한 부상을 입은 것으로 전해졌다'

print(tokenizer.morphs(kor_text))

In [38]:
#step1
from konlpy.tag import Hannanum
tokenizer = Hannanum()

kor_text = '밤에 귀가하던 여성에게 범죄를 시도한 대 남성이 구속됐다서울 제주경찰서는 \
            상해 혐의로 씨를 구속해 수사하고 있다고 일 밝혔다씨는 지난달 일 피해 여성을 \
            인근 지하철 역에서부터 따라가 폭행을 시도하려다가 도망간 혐의를 받는다피해 \
            여성이 저항하자 놀란 씨는 도망갔으며 신고를 받고 주변을 수색하던 경찰에 \
            체포됐다피해 여성은 이 과정에서 경미한 부상을 입은 것으로 전해졌다'

print(tokenizer.morphs(str(df['news'][3])))

['3', '서울뉴시스', '오동현', '기', '이', '자', '모바', '이', 'ㄹ', '게임', '기업', '컴투스', '는', '모바', '이', 'ㄹ', '야구', '게임', '컴투스프로', '야', '...', '3', '귀갓길', '여성', '을', '쫓', '아', '가', '아', '성범죄', '를', '시도', '하', 'ㄴ', '대', '남성', '이', '구속됐습니다서울', '강남경찰', '서는', '강간상', '...', '3', '아가타', '파리', '아가타', '파리', '아가타', '파리', '아가타', '파리', '아가타', '파리', '가', '시그니처', '라인', '...', 'Name', ':', 'news', ',', 'dtype', ':', 'object']


In [34]:
#step2
stopwords = ['에','는','은','을','했','에게',
             '있','이','의','하','한','다','과',
             '때문','할','수','무단','따른',
             '및','금지','전재','경향신문','기자',
             '는데','가','등','들','파이낸셜','저작','등','뉴스',
             'ㄴ', '어', '아', '고', '하', '자', '아며', '지', '...',
             ':', '으로', '보다', '도', '의',
             '는데', '도록', '면', '서는', 'ㄹ','ㅂ']

len(stopwords)

50

In [ ]:
# 토큰화 및 토큰화 과정에서 불용어를 제거하는 함수입니다.
def preprocessing(data):
    text_data = []
    
    for sentence in data:
        temp_data = []
        #- 토큰화
        temp_data = tokenizer.morphs(sentence)
        #- 불용어 제거
        temp_data = [word for word in temp_data if not word in stopwords]
        text_data.append(temp_data)
        
    text_data = list(map(''.join, text_data))
    
    return text_data

In [82]:
text_data = preprocessing(df['news'])
print(text_data[0])

안철수국민의당대표지나일서울영등포구이룸센터에서열리혁신준비위원회제차전체회의총선평가회에서마스크를벗안철수국민의당대표지나일서울영등포구이룸센터에서열리혁신준비위원회제차전체회의총선평가회에서마스크를벗있다파이낸셜뉴스안철수국민의당대표일제야당미래통합당과의연대의향대유연게대처겠다며가능성시사했다안대표이날라디오출연통합당과의정책연대가능성질문정책관철시키기위하어서ㄴ거기동의어떤당과도손잡어야것국회작동원리라며같이말했다안대표야당여러개으면훨씬더많국민의견제대로반영다며저희내안대여당동의여당손잡통과시키야당동의야당손잡관철시키것정상적방법라부연했다그무조건여당또는야당하고만연대ㄴ다는틀에서벗나겠다강조했다앞서알대표통합당포함야권합동총선평가회를제안바과거단순통합논의로는한계음지적한알대표야권전체혁신경쟁바뀌어야ㅁ강조기했다안대표자신대보수리더어되것이란전망나오것관련저야권정치인보수라말않아다반박했다진보되었든보수되었든저끊임없이책ㅁ많정부여당대비판적의견견지었다며보수정당집권었을때야당으로서비판진보라지금같형태에서정부를비판보수라ㄴ다저변하것없설명했다안대표초시ㅁ변하않았다며그러진심제대로전달되못하었다는것저실력부족해서제대로진심전달되노력어야겠다는생각라말었다저작권자파이낸셜뉴스전재재배포


In [83]:
#3.10
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 3)

print('훈련용 뉴스 기사의 개수 :', len(X_train))
print('테스트용 뉴스 기사의 개수 :', len(X_test))
print('훈련용 레이블의 개수 :', len(y_train))
print('테스트용 레이블의 개수 :', len(y_test))

훈련용 뉴스 기사의 개수 : 208
테스트용 뉴스 기사의 개수 : 70
훈련용 레이블의 개수 : 208
테스트용 레이블의 개수 : 70


In [84]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [54]:
def tfidf_vectorizer(data):
    data_counts = count_vect.transform(data)
    data_tfidf = tfidf_transformer.transform(data_counts)
    return data_tfidf

In [85]:
print(clf.predict(tfidf_vectorizer(text_data)))

['Urgent' 'Urgent' 'Urgent' 'Life/Culture' 'Urgent' 'Life/Culture'
 'Life/Culture' 'Urgent' 'Urgent' 'Urgent' 'Urgent' 'Urgent' 'Urgent'
 'Urgent' 'Life/Culture' 'Life/Culture' 'Life/Culture' 'Urgent'
 'Life/Culture' 'Urgent' 'Urgent' 'Urgent' 'Urgent' 'Life/Culture'
 'Life/Culture' 'Life/Culture' 'Urgent' 'Urgent' 'Urgent' 'Urgent'
 'Urgent' 'Life/Culture' 'Urgent' 'Urgent' 'Urgent' 'Urgent' 'Urgent'
 'Urgent' 'Life/Culture' 'Urgent' 'Life/Culture' 'Urgent' 'Urgent'
 'Life/Culture' 'Urgent' 'Urgent' 'Life/Culture' 'Urgent' 'Urgent'
 'Urgent' 'Life/Culture' 'Urgent' 'Urgent' 'Urgent' 'Urgent' 'Urgent'
 'Urgent' 'Urgent' 'Life/Culture' 'Economy' 'Economy' 'Life/Culture'
 'Economy' 'Economy' 'Life/Culture' 'Life/Culture' 'Life/Culture'
 'Life/Culture' 'Economy' 'Economy' 'Economy' 'Economy' 'Economy'
 'Life/Culture' 'Life/Culture' 'Economy' 'Economy' 'Economy'
 'Life/Culture' 'Economy' 'Economy' 'Economy' 'Life/Culture'
 'Life/Culture' 'Life/Culture' 'Economy' 'Economy' 'Economy' 'Econom

In [86]:
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     Economy       0.00      0.00      0.00        18
  IT/Science       0.00      0.00      0.00        12
Life/Culture       0.13      1.00      0.23         9
     Society       0.00      0.00      0.00        14
      Urgent       0.00      0.00      0.00        16
       World       0.00      0.00      0.00         1

    accuracy                           0.13        70
   macro avg       0.02      0.17      0.04        70
weighted avg       0.02      0.13      0.03        70



/home/aiffel0034/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#step3
#df2를 수집하고 DataFrame을 밑에 이어버리는 방식으로 df를 점점 연장해나가면 됨